# Multimodal Gömme ile Sanatsal Stillerin Analizi

*Yazar: [Jacob Marks](https://huggingface.co/jamarks)*
*Çevirmen: [Aylin Gümüş](https://huggingface.co/aylingumus)*

![Art Analysis Cover Image](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_cover_image.jpg)

Görüntüler gibi görsel veriler bilgi açısından inanılmaz derecede zengindir, ancak yapısal olmayan doğası gereği analiz etmesi zordur.

Bu not defterinde, multimodal embedding yöntemlerini ve hesaplanan öznitelikleri kullanarak resimlerdeki sanatsal stillerin nasıl analiz edileceğini keşfedeceğiz. Veri analizi ve görselleştirme için kullanmak üzere FiftyOne'a yükleyeceğimiz 🤗 Hub'dan [WikiArt veri kümesini](https://huggingface.co/datasets/huggan/wikiart) kullanacağız. Verilere birçok farklı açıdan göz atacağız:

- **Görüntü Benzerliği Arama ve Anlamsal (Semantik) Arama**: 🤗 Transformers'tan ön-eğitimli [CLIP](https://huggingface.co/openai/clip-vit-base-patch32) modelini kullanarak multimodal embedding'leri üreteceğiz ve yapısal olmayan aramalara izin vermek için veriyi indeksleyeceğiz.

- **Kümeleme ve Görselleştirme**: Embedding'leri kullanarak görüntüleri sanatsal stillerine göre kümeleyeceğiz ve UMAP boyut indirgeme yöntemini kullanarak sonuçları görselleştireceğiz.

- **Benzersizlik Analizi**: Veri kümesindeki diğer görüntülere ne kadar benzediğine bağlı olarak her görüntü için benzersizlik skoru atamak için embedding'lerimizi kullanacağız.

- **Görüntü Kalitesi Analizi**: Her bir görüntü için parlaklık, kontrast ve doygunluk gibi görüntü kalite metriklerini hesaplayacağız ve bu metriklerin, görüntülerin sanatsal stili ile nasıl ilişkili olduğunu göreceğiz.

## Hadi başlayalım! 🚀

Bu not defterini çalıştırmak için, aşağıdaki kütüphaneleri yüklemelisiniz:

In [ ]:
!pip install -U transformers huggingface_hub fiftyone umap-learn

İndirmeleri ışık hızında yapmak için [HF Transfer](https://pypi.org/project/hf-transfer/) yükleyin:

```bash
pip install hf-transfer
```

Ve `HF_HUB_ENABLE_HF_TRANSFER` ortam değişkenini ayarlayarak etkinleştirin:

```bash
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
```

<div class="alert alert-block alert-info">
<b>Not:</b> Bu not defteri <code>transformers==4.40.0</code>, <code>huggingface_hub==0.22.2</code> ve <code>fiftyone==0.23.8</code> ile test edilmiştir.
</div>

Şimdi bu not defteri için gerekli olan modülleri içe aktaralım:

In [1]:
import fiftyone as fo # base library and app
import fiftyone.zoo as foz # zoo datasets and models
import fiftyone.brain as fob # ML routines
from fiftyone import ViewField as F # for defining custom views
import fiftyone.utils.huggingface as fouh # for loading datasets from Hugging Face

Hub'dan 🤗 FiftyOne'a WikiArt veri kümesini yükleyerek başlayacağız. Bu veri kümesi Hugging Face'in `datasets` kütüphanesi aracılığıyla da yüklenebilir, ancak biz verileri doğrudan Datasets sunucusundan almak için [FiftyOne'ın 🤗 Hub entegrasyonunu](https://docs.voxel51.com/integrations/huggingface.html#huggingface-hub) kullanacağız. Hesaplamaları hızlı yapmak için, sadece ilk $1,000$ örneği indireceğiz.

In [2]:
dataset = fouh.load_from_hub(
    "huggan/wikiart", ## repo_id
    format="parquet", ## for Parquet format
    classification_fields=["artist", "style", "genre"], # columns to store as classification fields
    max_samples=1000, # number of samples to load
    name="wikiart", # name of the dataset in FiftyOne
)

Veri kümesinin ne içerdiğini görmek için özetini ekrana yazdırın:

In [3]:
print(dataset)

Name:        wikiart
Media type:  image
Num samples: 1000
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    artist:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    style:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    genre:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    row_idx:  fiftyone.core.fields.IntField


Veri kümesini [FiftyOne uygulamasında](https://docs.voxel51.com/user_guide/app.html) görselleştirin:

In [ ]:
session = fo.launch_app(dataset)

![WikiArt Dataset](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_wikiart_dataset.jpg)

Hadi, stillerini analiz edeceğimiz artistlerin isimlerini listeleyelim:

In [4]:
artists = dataset.distinct("artist.label")
print(artists)

['Unknown Artist', 'albrecht-durer', 'boris-kustodiev', 'camille-pissarro', 'childe-hassam', 'claude-monet', 'edgar-degas', 'eugene-boudin', 'gustave-dore', 'ilya-repin', 'ivan-aivazovsky', 'ivan-shishkin', 'john-singer-sargent', 'marc-chagall', 'martiros-saryan', 'nicholas-roerich', 'pablo-picasso', 'paul-cezanne', 'pierre-auguste-renoir', 'pyotr-konchalovsky', 'raphael-kirchner', 'rembrandt', 'salvador-dali', 'vincent-van-gogh']


## Benzer Sanat Eserlerini Bulma

Hoşunuza giden bir sanat eseri bulduğunuzda, ona benzer parçalar bulmak istemeniz doğaldır. Bunu vektör embedding'leriyle yapabiliriz! Dahası, multimodal embedding'leri aracılığıyla, bir tablonun ya da bir şiirin bile tanımı olabilecek, verilen bir metin sorgusunu en yakın temsil eden tabloları bulma yeteğini açacağız.

Hadi, görüntüler için Transformers'tan 🤗 ön-eğitimli bir CLIP Vision Transformer (ViT) modelini kullanarak multimodal embedding'leri üretelim. [FiftyOne Brain](https://docs.voxel51.com/user_guide/brain.html)'in `compute_similarity()` fonksiyonunu çalıştırmak, bu embedding'leri hesaplayacak ve bu embedding'leri, veri setinde bir benzerlik indeksi üretmek için kullanacak.

In [5]:
fob.compute_similarity(
    dataset, 
    model="zero-shot-classification-transformer-torch", ## type of model to load from model zoo
    name_or_path="openai/clip-vit-base-patch32", ## repo_id of checkpoint
    embeddings="clip_embeddings", ## name of the field to store embeddings
    brain_key="clip_sim", ## key to store similarity index info
    batch_size=32, ## batch size for inference
    )

Computing embeddings...
 100% |███████████████| 1000/1000 [5.0m elapsed, 0s remaining, 3.3 samples/s]    


<div style="padding: 10px; border-left: 5px solid #0078d4; font-family: Arial, sans-serif; margin: 10px 0;">

Alternatif olarak, modeli doğrudan 🤗 Transformers kütüphanesinden yükleyebilir ve modeli doğrudan kullanabilirsiniz:

```python
from transformers import CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
fob.compute_similarity(
    dataset, 
    model=model,
    embeddings="clip_embeddings", ## name of the field to store embeddings
    brain_key="clip_sim" ## key to store similarity index info
)
```

Bu ve daha fazlası hakkında daha kapsamlı bir rehber için, <a href="https://docs.voxel51.com/integrations/huggingface.html#transformers-library">FiftyOne'ın 🤗 Transformers entegrasyonunu</a> inceleyin.
</div>


FiftyOne uygulamasını yenileyin, örnek listedeki bir görüntünün onay kutusunu seçin ve veri kümesindeki en benzer görüntüleri görmek için fotoğraf ikonuna tıklayın. Arka planda, bu butona tıklamak, seçilen görüntüye en çok benzeyen görüntüleri bulan benzerlik indeksi, önceden hesaplanmış embedding'lere dayanarak bir sorguyu tetikler ve bunları uygulamada gösterir.

![Image Similarity Search](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_image_search.gif)

Bunu, verilen sanat eserine hangi sanat eserlerinin en çok benzediğini görmek için kullanabiliriz. Benzer sanat eserlerini bulmak (kullanıcılara önermek ya da bir koleksiyona eklemek için) ya da yeni bir eser için ilham almak amacıyla faydalı olabilir.

Ama dahası var! CLIP multimodal olduğu için, anlamsal (semantik) aramalar gerçekleştirmek için de kullanabiliriz! Bu, metin sorgularına dayalı görüntü araması yapabileceğimiz anlamına geliyor. Örneğin, "pastel ağaçlar" metniyle arama yapabiliriz ve veri kümesinde bu sorguya benzeyen tüm görüntüleri görebiliriz. Bunu yapmak için FiftyOne uygulamasındaki arama butonuna tıklayın ve bir metin sorgusu girin:

![Semantic Search](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_semantic_search.gif)

Sahne arkasında, metin tokenize edilir, CLIP'in metin kodlayıcısı ile gömülür, ardından veri kümesindeki en benzer görüntüleri bulmak için benzerlik indeksini sorgulamak için kullanılır. Bu, metin sorgularına dayalı görüntü aramak için oldukça etkili bir yöntemdir ve belirli bir tema ya da stille eşleşen görüntüleri bulmak için faydalı olabilir. Ve bu CLIP ile sınırlı değil; görüntüler ve metin için embedding'ler üretebilen 🤗 Transformers'tan herhangi bir CLIP-benzeri model kullanabilirsiniz!

<div class="alert alert-block alert-info">
💡 Etkili vektör araması ve büyük veri kümeleri üzerinde indeksleme için, Fifty One'ın <a href="https://voxel51.com/vector-search">açık kaynak vektör veritabanları ile yerel entegrasyonları</a> var.
</div>


## Kümeleme ve Görselleştirme ile Sanatsal Motiflerin Ortaya Çıkarılması

Benzerlik ve anlamsal (semantik) aramalar yaparak verilerle daha etkili bir şekilde etkileşim kurmaya başlayabiliriz. Ancak bunu bir adım daha ileri götürebilir ve işin içine biraz da denetimsiz öğrenme ekleyebiliriz. Bu, WikiArt veri kümesindeki sanatsal paternleri belirlememize yardımcı olacak; stilistik, topikal ve hatta kelimelerle ifade edilmesi zor olan motifleri bile.

Bunu iki yöntemle yapacağız:

1. **Boyut İndirgeme**: Embedding'lerin boyutunu 2B'ye indirgemek ve verileri bir dağılım grafiğinde görselleştirmek için UMAP yöntemini kullanacağız. Bu, görüntülerin stillerine, türlerine ve artistlerine göre nasıl kümelendiğini görmemizi sağlayacaktır.
2. **Kümeleme**: Görüntüleri embedding'lerine göre kümelemek ve hangi grupların ortaya çıktığını görmek için K-Means kümeleme yöntemini kullanacağız.

Boyut indirgeme için, önceden hesaplanmış embedding'leri parametre olarak geçerek FiftyOne Brain'den `compute_visualization()` fonksiyonunu çalıştıracağız. Boyut indirgeme amacıyla UMAP  yöntemini kullanmak için `method="umap"` olarak belirtiyoruz, ancak PCA veya t-SNE de kullanabiliriz:

In [6]:
fob.compute_visualization(dataset, embeddings="clip_embeddings", method="umap", brain_key="clip_vis")

Generating visualization...


/opt/homebrew/Caskroom/miniforge/base/envs/fdev/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


UMAP( verbose=True)
Tue Apr 30 11:51:45 2024 Construct fuzzy simplicial set
Tue Apr 30 11:51:46 2024 Finding Nearest Neighbors
Tue Apr 30 11:51:47 2024 Finished Nearest Neighbor Search
Tue Apr 30 11:51:48 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Tue Apr 30 11:51:49 2024 Finished embedding


Şimdi FiftyOne uygulamasında veri kümesindeki her görüntü için 2B bir nokta göreceğimiz bir panel açabiliriz. Bu niteliklerin görüntü özelliklerimiz tarafından ne kadar güçlü bir şekilde yakalandığını görmek için, veri kümesindeki noktaları artist veya tür gibi alanlara göre renklendirebiliriz:

![UMAP Visualization](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_visualize_embeddings.gif)

Benzer görüntüleri bir arada gruplayabilmek için embedding'ler üzerinde kümeleme de yapabiliriz — belki bu sanat eserlerinin dominant özellikleri halihazırda var olan etiketler tarafından algılanmamıştır ya da belki tanımlamak istediğimiz farklı alt türler vardır. Verilerimizi kümelemek için [FiftyOne Clustering Plugin](https://github.com/jacobmarks/clustering-plugin)'i indirmemiz gerekecek:

In [ ]:
!fiftyone plugins download https://github.com/jacobmarks/clustering-plugin

Uygulamayı tekrar yenileyerek, kümeleme fonksiyonalitesine uygulamadaki bir operatör yardımıyla erişebiliriz. Operatör listesini açmak için ters tırnak tuşuna basın, "cluster" yazın ve açılan menüden operatörü seçin. Bu, kümeleme algoritmasını, hiperparametreleri ve kümelenecek alanı belirleyebileceğimiz etkileşimli bir panel açacaktır. Basit tutmak için, $10$ küme ile K-Means kümelemesini kullanacağız:

Böylelikle kümeleri uygulamada görselleştirebilir ve görüntülerin embedding'lerine göre nasıl gruplandığını görebiliriz:

![K-means Clustering](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_clustering.gif)

Bazı kümelerin artisti, bazılarının ise türü veya stili temel aldığını görebiliriz. Diğerleri ise daha soyut ve veriden ilk bakışta anlaşılamayan alt türleri veya diğer gruplamaları temsil edebilir.

## En Benzersiz Sanat Eserlerinin Belirlenmesi

Veri kümemiz hakkında sorabileceğimiz ilginç sorulardan biri, her görüntünün ne kadar eşsiz olduğudur. Bu soru, benzer görüntüleri önerilmesi, kopyaların tespit edilmesi veya aykırı değerlerin belirlenmesi gibi birçok uygulama için önemlidir. Sanat eseri bağlamında, bir tablonun ne kadar eşsiz olduğu, onun değerini belirlemede önemli bir faktör olabilir.

Benzersizliği tanımlamanın milyonlarca yolu olsa da, görüntü embedding'lerimiz, her örneğe veri kümesindeki diğer örneklere ne kadar benzer olduğuna göre nicel olarak bir benzersizlik skoru atamamızı sağlar. Açıkçası, FiftyOne Brain'in `compute_uniqueness()` fonksiyonu, her örneğin embedding'i ile en yakın komşuları arasındaki mesafeye bakar ve bu mesafeye göre $0$ ile $1$ arasında bir skor hesaplar. Skorun $0$ olması, örneğin belirgin olmayan veya diğerlerine çok benzeyen bir örnek olduğunu; $1$ olması ise çok benzersiz bir örnek olduğunu gösterir.

In [7]:
fob.compute_uniqueness(dataset, embeddings="clip_embeddings") # compute uniqueness using CLIP embeddings

Computing uniqueness...
Uniqueness computation complete


Böylece embedding panelinde buna göre renklendirme yapabilir, benzersizlik skoruna göre filtreleyebilir ve hatta sıralayarak veri kümesindeki en benzersiz görüntüleri görebiliriz.

In [ ]:
most_unique_view = dataset.sort_by("uniqueness", reverse=True)
session.view = most_unique_view.view() # Most unique images

![Most Unique Images](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_most_unique.jpg)

In [ ]:
least_unique_view = dataset.sort_by("uniqueness", reverse=False)
session.view = least_unique_view.view() # Least unique images

![Least Unique Images](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_least_unique.jpg)

Bir adım daha ileriye giderek, hangi artistin en benzersiz eserleri üretme eğiliminde olduğu sorusunu da cevaplayabiliriz. Her artistin tüm sanat eserleri için ortalama benzersizlik skorunu hesaplayabiliriz.

In [8]:
artist_unique_scores = {
    artist: dataset.match(F("artist.label") == artist).mean("uniqueness")
    for artist in artists
}

sorted_artists = sorted(
    artist_unique_scores, key=artist_unique_scores.get, reverse=True
)

for artist in sorted_artists:
    print(f"{artist}: {artist_unique_scores[artist]}")

Unknown Artist: 0.7932221632002723
boris-kustodiev: 0.7480731948424676
salvador-dali: 0.7368807620414014
raphael-kirchner: 0.7315448102204755
ilya-repin: 0.7204744626806383
marc-chagall: 0.7169373812321908
rembrandt: 0.715205220292227
martiros-saryan: 0.708560775790436
childe-hassam: 0.7018343391132756
edgar-degas: 0.699912746806587
albrecht-durer: 0.6969358680800216
john-singer-sargent: 0.6839955708720844
pablo-picasso: 0.6835137858302969
pyotr-konchalovsky: 0.6780653000855895
nicholas-roerich: 0.6676504687452387
ivan-aivazovsky: 0.6484361530090199
vincent-van-gogh: 0.6472004520699081
gustave-dore: 0.6307283287457358
pierre-auguste-renoir: 0.6271467146993583
paul-cezanne: 0.6251076007168186
eugene-boudin: 0.6103397516167454
camille-pissarro: 0.6046182609119615
claude-monet: 0.5998234558947573
ivan-shishkin: 0.589796389836674


Görünüşe göre, veri kümemizdeki en benzersiz eserlere sahip artist Boris Kustodiev! Hadi bazı eserlerine göz atalım:

In [10]:
kustodiev_view = dataset.match(F("artist.label") == "boris-kustodiev")
session.view = kustodiev_view.view()

![Boris Kustodiev Artwork](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_kustodiev_view.jpg)

## Sanat Eserinin Görsel Özelliklerle Karakterize Edilmesi

Her şeyi toparlayacak olursak, temel konulara geri dönelim ve veri kümemizdeki görüntülerin bazı temel özelliklerini analiz edelim. Her bir görüntü için parlaklık, kontrast ve doygunluk gibi standart metrikleri hesaplayacağız ve bu metriklerin sanat eserlerinin artistik stili ve türü ile nasıl ilişkili olduğunu göreceğiz.

Bu analizleri yapmak için, [FiftyOne Image Quality Plugin](https://github.com/jacobmarks/image-quality-issues)'ini indirmemiz gerekecek:

In [ ]:
!fiftyone plugins download https://github.com/jacobmarks/image-quality-issues/

Uygulamayı yenileyin ve operatör listesini tekrar açın. Bu kez `compute` yazın ve görüntü kalitesi operatörlerinden birini seçin. Parlaklık ile başlayacağız:

![Compute Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_compute_brightness.gif)

Operatör çalışmayı tamamladığında, her bir görüntü için parlaklık skoru içeren yeni bir alan veri kümemize eklenmiş olacak. Böylece bu veriyi uygulamada görselleştirebiliriz:

![Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_brightness.gif)

Aynı zamanda parlaklığa göre renklendirebilir ve hatta stil gibi veri kümesindeki diğer alanlarla nasıl ilişkili olduğunu da görebiliriz:

![Style by Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_style_by_brightness.gif)

Şimdi aynı işlemi kontrast ve doygunluk için de yapalım. İşte doygunluk için elde edilen sonuçlar:

![Filter by Saturation](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_filter_by_saturation.jpg)

Umarız bu, verilerinize derin sinir ağları uygulamanın her şey demek olmadığını gösteriyordur. Bazen basit metrikler de en az onlar kadar bilgi verebilir ve verinize farklı bir bakış açısı kazandırabilir 🤓!

<div class="alert alert-block alert-info">
📚 Daha büyük veri kümeleri için, işlemleri daha sonra yürütülmek üzere <a href="https://docs.voxel51.com/plugins/using_plugins.html#delegated-operations">devretmeyi</a> düşünebilirsiniz.
</div>

## Sırada Ne Var?

Bu not defterinde, görüntülerdeki artistik stilleri analiz etmek için, multimodal embedding'leri nasıl kullanacağımızı, gözetimsiz öğrenmeyi ve geleneksel görüntü işleme tekniklerini keşfettik. Görüntü benzerliği ve anlamsal (semantik) arama işlemlerini nasıl gerçekleştirebileceğimizi, görüntüleri stillerine göre nasıl kümeleyeceğimizi, görüntülerin benzersizliğini nasıl analiz edeceğimizi ve görüntü kalite metriklerini nasıl hesaplayacağımızı gördük. Bu teknikler, sanat eseri koleksiyonlarından tıbbi görüntülere veya uydu görüntülerine kadar çok çeşitli görsel veri kümelerine uygulanabilir. [Hugging Face Hub üzerinden farklı bir veri kümesi yüklemeyi deneyin](https://docs.voxel51.com/integrations/huggingface.html#loading-datasets-from-the-hub) ve veriye hangi farklı bakış açılarıyla yaklaşabileceğinizi keşfedin!

Daha da ileri gitmek isterseniz, aşağıdaki bazı ek analizleri de deneyebilirsiniz:

- **Sıfır-Atış Sınıflandırma (Zero-Shot Classification)**: 🤗 Transformers tarafından önceden eğitilmiş bir görsel-dil modelini kullanarak, herhangi bir eğitim verisine ihtiyaç duymadan, veri kümesindeki görüntüleri konuya ya da içeriğe göre sınıflandırabilirsiniz. Daha fazla bilgi için [Zero-Shot Classification eğitimi](https://docs.voxel51.com/tutorials/zero_shot_classification.html)'ne göz atın.
- **Görüntü Açıklama (Image Captioning)**: 🤗 Transformers tarafından önceden eğitilmiş bir görsel-dil modelini kullanarak, veri kümesindeki görseller için açıklamalar (caption) oluşturabilirsiniz. Bu açıklamaları konu modellemesinde kullanabilir veya bu açıklamaların embedding'lerine göre sanat eserlerini kümelendirebilirsiniz. Daha fazla bilgi için FiftyOne'ın [Image Captioning Plugin](https://github.com/jacobmarks/fiftyone-image-captioning-plugin)'ine göz atın.

### 📚 Kaynaklar

- [FiftyOne 🤝 🤗 Hub Integration](https://docs.voxel51.com/integrations/huggingface.html#huggingface-hub)
- [FiftyOne 🤝 🤗 Transformers Integration](https://docs.voxel51.com/integrations/huggingface.html#transformers-library)
- [FiftyOne Vector Search Integrations](https://voxel51.com/vector-search/)
- [Visualizing Data with Dimensionality Reduction Techniques](https://docs.voxel51.com/tutorials/dimension_reduction.html)
- [Clustering Images with Embeddings](https://docs.voxel51.com/tutorials/clustering.html)
- [Exploring Image Uniqueness with FiftyOne](https://docs.voxel51.com/tutorials/uniqueness.html)

## FiftyOne Açık Kaynak Projesi

[FiftyOne](https://github.com/voxel51/fiftyone/) yüksek kalitede veri kümeleri ve bilgisayarla görme modelleri oluşturmak için kullanılan önde gelen açık kaynak araç setidir. 2 milyondan fazla kez indirilen FiftyOne, dünya genelindeki geliştiriciler ve araştırmacılar tarafından güvenle kullanılmaktadır.

💪 FiftyOne ekibi, açık kaynak topluluğundan gelen katkıları memnuniyetle karşılar! Eğer FiftyOne'a katkıda bulunmak istiyorsanız, [katkı rehberine](https://github.com/voxel51/fiftyone/blob/develop/CONTRIBUTING.md) göz atabilirsiniz.